# Vehicle Classification

## Evaluate your model


In [1]:
import os

from sklearn.metrics import accuracy_score, classification_report

from utils import utils
from models import resnet_50


2022-11-16 05:59:56.218281: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-16 05:59:56.218320: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Experiment settings

Set here the two variables in the following way:

- **CONFIG_YML:** assign the path to the config.yml file used for the experiment you want to evaluate
- **WEIGHTS:** assign the path to the model weights (.h5 file) you want to evaluate


In [2]:
# Modify this
CONFIG_YML = "../experiments/exp_006/config.yml"
WEIGHTS = "../experiments/exp_006/model.67-2.4287.h5"

In [3]:
# (!) Don't touch this cell, will load the config file specified before
config = utils.load_config(CONFIG_YML)
config

{'seed': 123,
 'data': {'directory': '/home/app/src/data/car_ims_v1/train',
  'labels': 'inferred',
  'label_mode': 'categorical',
  'validation_split': 0.2,
  'image_size': [224, 224],
  'batch_size': 32},
 'model': {'weights': '/home/app/src/experiments/exp_005/model.69-2.9774.h5',
  'input_shape': [224, 224, 3],
  'classes': 196,
  'dropout_rate': 0.3,
  'data_aug_layer': {'random_flip': {'mode': 'horizontal'},
   'random_rotation': {'factor': 0.2},
   'random_zoom': {'height_factor': 0.2, 'width_factor': 0.2}}},
 'compile': {'optimizer': {'sgd': {'learning_rate': 5e-05}},
  'loss': 'categorical_crossentropy',
  'metrics': ['accuracy']},
 'fit': {'epochs': 100,
  'callbacks': {'early_stopping': {'monitor': 'val_accuracy',
    'patience': 5,
    'restore_best_weights': True},
   'model_checkpoint': {'filepath': '/home/app/src/experiments/exp_006/model.{epoch:02d}-{val_loss:.4f}.h5',
    'save_best_only': True},
   'tensor_board': {'log_dir': '/home/app/src/experiments/exp_006/logs'}}

In [4]:
# (!) Don't touch this cell, will use the config file to infer the class names
#     and also to load the corresponding testing dataset.
#     If you get an error, you must review your data/code.

MODEL_CLASSES = utils.get_class_names(config)

if len(MODEL_CLASSES) != config['model']['classes']:
    raise ValueError(
        "Number of classes doesn't match between your model "
        "and your data!"
    )

_dirname, _ = os.path.split(config['data']['directory'])
TEST_FOLDER = os.path.join(_dirname, 'test')

if not os.path.exists(TEST_FOLDER):
    raise ValueError("'test' folder not found!")
    
if len(os.listdir(TEST_FOLDER)) != config['model']['classes']:
    raise ValueError(
        "Number of classes doesn't match between your model "
        "and your testing dataset!"
    )
    
if set(os.listdir(TEST_FOLDER)) != set(MODEL_CLASSES):
    raise ValueError(
        "The name of the subfolders inside your test set "
        "doesn't match with the model classes!"
    )

## Load your model

Use `resnet_50.create_model()` and remember to properly setup the model weights!

Assign the model to the variable `cnn_model`.


In [5]:
# Complete this
# TODO
cnn_model = resnet_50.create_model(weights=WEIGHTS)

# It should print your model correctly
print(cnn_model.summary())

2022-11-16 05:59:58.716460: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-16 05:59:58.716502: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-16 05:59:58.716519: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (wolffgang): /proc/driver/nvidia/version does not exist
2022-11-16 05:59:58.717983: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                             

## Get predictions from testing dataset

In [6]:
# (!) Don't touch this cell, will use the loaded model and
#     the function utils.predict_from_folder() to get 
#     model predictions and the corresponding true labels
#     so we can measure the accuracy

predictions, labels = utils.predict_from_folder(
    folder=TEST_FOLDER, 
    model=cnn_model, 
    input_size=config["data"]["image_size"], 
    class_names=MODEL_CLASSES,
)

if len(predictions) != len(labels):
    raise ValueError(
        "The lenght of predictions and labels lists doesn't match!"
    )

if not isinstance(predictions[0], str):
    raise ValueError(
        "Model predictions should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )

if not isinstance(labels[0], str):
    raise ValueError(
        "Ground true labels should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )


In [7]:
# (!) Don't touch this cell

print(classification_report(y_true=labels, y_pred=predictions))


                                                        precision    recall  f1-score   support

                            AM General Hummer SUV 2000       0.84      0.70      0.77        44
                             Acura Integra Type R 2001       0.59      0.39      0.47        44
                                   Acura RL Sedan 2012       0.12      0.03      0.05        32
                                   Acura TL Sedan 2012       0.26      0.26      0.26        43
                                  Acura TL Type-S 2008       0.23      0.21      0.22        42
                                  Acura TSX Sedan 2012       0.67      0.05      0.09        40
                              Acura ZDX Hatchback 2012       0.24      0.18      0.21        39
              Aston Martin V8 Vantage Convertible 2012       0.41      0.27      0.32        45
                    Aston Martin V8 Vantage Coupe 2012       0.27      0.10      0.14        41
                  Aston Martin Virage C

In [8]:
predictions

['Bugatti Veyron 16.4 Convertible 2009',
 'Bentley Continental Supersports Conv. Convertible 2012',
 'Bugatti Veyron 16.4 Convertible 2009',
 'Bugatti Veyron 16.4 Convertible 2009',
 'Bugatti Veyron 16.4 Convertible 2009',
 'Jaguar XK XKR 2012',
 'Bugatti Veyron 16.4 Convertible 2009',
 'Bugatti Veyron 16.4 Convertible 2009',
 'Bugatti Veyron 16.4 Convertible 2009',
 'Chevrolet Corvette Ron Fellows Edition Z06 2007',
 'Bugatti Veyron 16.4 Convertible 2009',
 'Bentley Continental GT Coupe 2012',
 'Daewoo Nubira Wagon 2002',
 'Bugatti Veyron 16.4 Convertible 2009',
 'Bugatti Veyron 16.4 Convertible 2009',
 'Bugatti Veyron 16.4 Convertible 2009',
 'Suzuki Kizashi Sedan 2012',
 'Bugatti Veyron 16.4 Convertible 2009',
 'BMW M6 Convertible 2010',
 'BMW M6 Convertible 2010',
 'Bugatti Veyron 16.4 Convertible 2009',
 'Bugatti Veyron 16.4 Convertible 2009',
 'Bugatti Veyron 16.4 Convertible 2009',
 'Bugatti Veyron 16.4 Convertible 2009',
 'MINI Cooper Roadster Convertible 2012',
 'Bugatti Veyro

In [9]:
# (!) Don't touch this cell

acc = accuracy_score(y_true=labels, y_pred=predictions)

print(f"Your model accuracy is {acc:.4f}!")

if acc < .3:
    raise ValueError("Your model accuracy is too low :(\nYou can do it better! :)")


Your model accuracy is 0.3557!
